In [1]:
import pandas as pd
import xlwings as xw
from xlwings.constants import LineStyle

from tkinter import *
from PIL import ImageTk, Image
from tkinter.ttk import Combobox
from tkinter import filedialog
from tkinter import ttk

In [2]:
# Allgemeine Funktionen
def open_closed_or_opened_file(path):
    """ Mit der Funktion wird zuerst versucht, die Excel-Datei mit pandas zu öffnen.
        Falls die Datei bereits von einem User geöffnet ist, wird eine Kopie erstellt.
    """
    date_format = '%d.%m.%Y'
    try:
        df = pd.read_excel(path, decimal=',')
    except:
        if '~$' in path:
            path = path.replace('~$', '')
        temp_dir = tempfile.mkdtemp()        
        temp_file_path = tempfile.mktemp(suffix='.xlsx', dir=temp_dir)
        shutil.copy2(path, temp_file_path)
        try:
            df = pd.read_excel(temp_file_path, decimal=',')  # Die temporäre Datei wird hier geöffnet
        finally:
            shutil.rmtree(temp_dir)
    return df

In [3]:
# Funktion für Go-Button
def go_dodo():
    global excel, fertig, delete_cols
    # Entfernen reservierter Proben
    try:
        excel = excel[excel['reserved for'].isnull()]
    except:
        pass
    try:
        excel = excel[excel['CBH reserved for'].isnull()]
    except:
        pass
    
    # Entfernen nicht relevanter Daten
    delete_cols = [col for col in delete_cols if col not in save_cols]
    excel = excel.drop(columns=delete_cols, errors='ignore')
    
    # Entfernen der Duplikate
    excel = excel.drop_duplicates()
    excel = excel.reset_index().drop('index', axis=1)
    
#     # Proben ohne Referenzbereich markieren
#     index_ref = []
#     for i in range(len(excel)):
#         if ('not specified' in str(list(excel['Reference range / Cut-off'])[i])) | ('nan' in str(list(excel['Reference range / Cut-off'])[i])):
#             index_ref.append(i+11)
            
    # Titerangabe konvertieren
    excel['Titer M'] = ['']*len(excel)
    excel['Titer R'] = ['']*len(excel)
    titer_index = []
    for i in range(len(excel)):    
        if ':' in str(excel['Measured value'][i]):
            titer_index.append(i)
            if '<' in str(excel['Measured value'][i]):
                excel['Titer M'][i] = excel['Measured value'][i].split('<')[-1].strip()
            else:
                excel['Titer M'][i] = excel['Measured value'][i].split(':')[-1].strip()
        elif '.' in str(excel['Measured value'][i]):
            excel['Titer M'][i] = str(excel['Measured value'][i]).replace('.', ',')
        if ':' in str(excel['Reference range / Cut-off'][i]):
            if '<' in str(excel['Reference range / Cut-off'][i]):
                excel['Titer R'][i] = excel['Reference range / Cut-off'][i].split('<')[-1].strip()
            else:
                excel['Titer R'][i] = excel['Reference range / Cut-off'][i].split(':')[-1].strip()
                
    # Abgleich Messwert
    excel['Abgleich Messwert'] = ['']*len(excel)
    excel['Titer'] = ['']*len(excel)
    
    for i in titer_index:
        excel['Titer'][i] = 'x'
        try:
            if int(excel['Titer M'][i]) > int(excel['Titer R'][i]):
                excel['Abgleich Messwert'][i] = 'positiv'
                if int(excel['Titer M'][i]) < 2 * int(excel['Titer R'][i]):
                    excel['Abgleich Messwert'][i] = 'niedrig positiv'
                elif int(excel['Titer M'][i]) >= 5 * int(excel['Titer R'][i]):
                    excel['Abgleich Messwert'][i] = 'hoch positiv'
            elif int(excel['Titer M'][i]) < int(excel['Titer R'][i]):
                excel['Abgleich Messwert'][i] = 'negativ'
        except:
            pass

    # Kürzen auf 100 gleichverteilte Proben pro Master ID
    # Subset für Master ID erstellen
    master_id = []
    for i in titer_index:
        if excel['Master ID'][i] not in master_id:
            master_id.append(excel['Master ID'][i])

    for master in master_id:
        subset = excel[excel['Master ID'] == master]
        subset = subset[subset['Titer']=='x']
        subset_niedrig = subset[subset['Abgleich Messwert']=='niedrig positiv'].head(30)
        subset_positiv = subset[subset['Abgleich Messwert']=='positiv'].head(30)
        subset_hoch = subset[subset['Abgleich Messwert']=='hoch positiv'].head(30)
        subset_negativ = subset[subset['Abgleich Messwert']=='niedrig positiv'].head(10)
        subset_final = pd.concat([subset_niedrig, subset_positiv, subset_hoch, subset_negativ])
        subset_final = subset_final.reset_index(drop=True)
        rows_to_drop = subset[~subset.index.isin(subset_final.index)].index
        excel = excel.drop(rows_to_drop)

    # hinzugefügte Spalten wieder rauslöschen
    excel = excel.drop(['Titer M', 'Titer R', 'Abgleich Messwert', 'Titer'], axis=1)
    
    # Vorlage einlesen
    vorlage = pd.read_excel("O:\Forschung & Entwicklung\Entwicklung\Software-Entwicklung\Verfügbarkeitslisten DM\Vorlage\Vorlage_Output.xlsx", sheet_name='Output')
    vorlage_spalten = list(vorlage.iloc[8,:])
    # excel bearbeiten
    excel = excel.rename(columns={'Total volume': 'Total volume [ml]'})
    excel_spalten = list(excel.columns)
    
    # Überprüfe, ob alle Spalten in der Liste im DataFrame vorhanden sind
    added_columns = [col for col in vorlage_spalten if col not in excel_spalten]
    # Spalten, die leer sind, wurden aus excel entfernt. Diese wieder hinzufügen
    for col in added_columns:
        if col not in excel_spalten:
            excel[col] = ['']*len(excel)
    # Füge fehlende Spalten am Ende der Liste hinzu
    added_columns = [col for col in vorlage_spalten if col not in list(excel.columns)]
    new_columns_order = vorlage_spalten + added_columns
    for col in save_cols:
        new_columns_order.append(col)
    # Wähle die Spalten des DataFrame in der neuen Reihenfolge aus
    excel = excel[new_columns_order]
    excel['price/ml'] = ['']*len(excel)
    
    # In Excel-Vorlage übertragen
    wb = xw.Book("O:\Forschung & Entwicklung\Entwicklung\Software-Entwicklung\Verfügbarkeitslisten DM\Vorlage\Vorlage_Output.xlsx")
    sheet = wb.sheets['Output']
    sheet.range('A11').value = excel.values
    
    # zusätzliche Spalten in Vorlage einfügen
    plus_cols = [col for col in list(excel.columns) if col not in vorlage_spalten]
    start_zelle = 'R10'
    referenz_zelle = sheet.range('Q10')
    # Einträge in Excel einfügen, jeweils in einer benachbarten Spalte ab der Startzelle
    for index, eintrag in enumerate(plus_cols):
        zelle = sheet.range(start_zelle).offset(column_offset=index)
        zelle.value = eintrag
        # Formatierung von Referenzzelle auf die eingefügte Zelle übertragen
        zelle.color = referenz_zelle.color
        zelle.font.size = referenz_zelle.font.size
        zelle.font.bold = referenz_zelle.font.bold
        # Rahmen von Referenzzelle auf die eingefügte Zelle übertragen
        zelle.api.Borders.LineStyle = LineStyle.xlContinuous
        zelle.api.Borders.Weight = 2
        
    # Proben ohne Referenzbereich markieren
#     for ref_i in index_ref:
#         cell = sheet.range('K{}'.format(ref_i))
#         cell.color = (127,255,212)
    for i in range(len(excel)):
        cell = sheet.range('K{}'.format(i+11))
        cell_value = cell.value
        if cell_value is None or 'not specified' in str(cell_value):
            cell.color = (127,255,212)
    
        
    # nach Parameter gruppieren
    group_para = excel.groupby('Laboratory parameter')['Total volume [ml]'].sum()
    group_para = pd.DataFrame(group_para)
    group_para.reset_index(inplace=True)
    group_para['Total volume [ml]'] = group_para['Total volume [ml]'].map(lambda x: x/1000)
    sheet_sum = wb.sheets['Zusammenfassung']
    sheet_sum.range('A4').value = group_para.values
    
    wb.save('Verfügbarkeitsliste.xlsx')
    
    
    
    leer = Label(root, text='', bg = '#eeeee4')
    leer.grid(row=19, column=1)
    fertig = Label(root, text = 'Fertig! #TeamDodo',bg = '#eeeee4', fg = '#869287',
                   font=('Ink free',10,'bold'))
    fertig.grid(row=20, column=1, columnspan=3)

In [4]:
# Funktion für Browse-Button
def select_file():
    global checkbox_vars, buttons, save_cols, delete_cols, excel
#     global delete_standard_cols
    
    try:
        fertig.grid_forget()
    except:
        pass
    
    def create_button(selected_element):
        if selected_element and selected_element not in [button['text'] for button in buttons]:
            button = Button(button_frame, text=selected_element,
                            bg='#DADEDB', font=('Ink free', 10),
                            command=lambda element=selected_element: remove_button(element))
            button.pack(anchor=W)
            buttons.append(button)
            save_cols.append(button.cget("text"))

            # Aktualisiere das Canvas-Scroll-Feld
            canvas.update_idletasks()
            canvas.config(scrollregion=canvas.bbox("all"))

    def remove_button(selected_element):
        for button in buttons:
            if button['text'] == selected_element:
                save_cols.remove(button.cget("text"))
                button.destroy()
                buttons.remove(button)
                # Entferne das Element auch aus der Dropdown-Liste
                delete_cols.append(selected_element)
                selected_element_dropdown['values'] = tuple(sorted(delete_cols))

    file_path = filedialog.askopenfilename(title="Datei auswählen")
    excel = open_closed_or_opened_file(file_path)
    excel = excel.dropna(axis=0, how='all')
    excel.columns = excel.iloc[0]
    excel = excel[1:]
    # Entfernen von leeren Spalten
    excel = excel.dropna(axis=1, how='all')

    leer = Label(root, text='', bg='#eeeee4')
    leer.grid(row=3, column=1, columnspan=3)

    text_daten = Label(root, text='2. Entfernen nicht relevanter Daten', bg='#eeeee4',
                       font=('Ink free', 11, 'bold'))
    text_daten.grid(row=4, column=1, columnspan=3)

#     standard_spalten = [
#         'Freigabe',
#         'Grund der nicht erhaltenen Freigabe',
#         'CBH',
#         'Sample ID',
#         'Volume',
#         'unit of quantity'
#     ]

    delete_cols = list(excel.columns)
#     alle_spalten = [spalte for spalte in alle_spalten if spalte not in standard_spalten]
    # Pflichtspalten aus der Liste entfernen
    pflicht_spalten = ['Master ID', 'Pat ID', 'Total volume', 'Sample condition', 'Matrix',
                      'Gender', 'Age of proband [years]', 'Laboratory parameter', 'Measured value',
                      'Unit', 'Reference range / Cut-off', 'Remarks on reference range',
                      'Test method', 'Analytical platform', 'Manufacturer of analytical platform',
                      'Test kit', 'Manufacturer of test kit', 'price/ml']
    delete_cols = [spalte for spalte in delete_cols if spalte not in pflicht_spalten]
    
#     delete_standard_cols = standard_spalten.copy()
    
#     text_entfernen = Label(root, text='Diese Spalten werden entfernt:', bg='#eeeee4',
#                                 font=('Ink free', 11))
#     text_entfernen.grid(row=5, column=1, columnspan=3)
    
#     def checkbox_selected(checkbox_value):
#         if checkbox_value in delete_standard_cols:
#             delete_standard_cols.remove(checkbox_value)
#         else:
#             delete_standard_cols.append(checkbox_value)

#     checkbox_vars = []

#     for index, spalte in enumerate(standard_spalten):
#         var = BooleanVar()
#         var.set(True)
#         checkbox = Checkbutton(root, text=spalte + ' ', variable=var,
#                                command=lambda spalte=spalte: checkbox_selected(spalte),
#                                bg='#eeeee4', font=('Ink free', 10))
#         checkbox.grid(row=5, column=4 + index, sticky=W)
#         checkbox_vars.append(var)

    text_mehr_entfernen = Label(root, text='Wähle hier bitte aus, falls du Spalten beibehalten möchtest', bg='#eeeee4',
                                font=('Ink free', 11))
    text_mehr_entfernen.grid(row=5, column=1, columnspan=7)

    selected_element_var = StringVar()
    selected_element_dropdown = Combobox(root, textvariable=selected_element_var, values=delete_cols,
                                          state='readonly', width=35)
    selected_element_dropdown.grid(row=6, column=1, padx=10, pady=10, columnspan=6)
    selected_element_dropdown.bind("<<ComboboxSelected>>", lambda event: create_button(selected_element_var.get()))

    buttons = []
    save_cols = []

    # Erstelle den Scrollbar-Frame
    scrollbar_frame = Frame(root, bg='#eeeee4')
    scrollbar_frame.grid(row=7, column=1, padx=5, pady=10, columnspan=5, sticky=N+S)

    # Erstelle eine vertikale Scrollbar
    scrollbar = Scrollbar(scrollbar_frame, orient=VERTICAL)
    scrollbar.pack(side=RIGHT, fill=Y)

    # Erstelle einen Canvas
    canvas = Canvas(scrollbar_frame, yscrollcommand=scrollbar.set, width=250, height=150, bg='#eeeee4')
    canvas.pack(side=LEFT, fill=BOTH, expand=True)

    # Konfiguriere die Scrollbar
    scrollbar.config(command=canvas.yview)

    # Erstelle einen Frame für die Buttons
    button_frame = Frame(canvas, bg='#eeeee4')
    canvas.create_window((0,0), window=button_frame, anchor=NW)

    # Füge dem Canvas einen Befehl zum Anpassen der Größe hinzu
    button_frame.bind("<Configure>", lambda event, canvas=canvas: canvas.configure(scrollregion=canvas.bbox("all")))

    leer = Label(root, text='', bg = '#eeeee4')
    leer.grid(row=17, column=1)
    text_go = Label(root, text='3. Erstellen der Verfügbarkeitsliste', bg='#eeeee4',
                    font=('Ink free', 11, 'bold'))
    text_go.grid(row=18, column=1, columnspan=3)
    go = Button(root, text='   GO   ', bg='#869287', font=('Ink free', 11, 'bold'), command=go_dodo)
    go.grid(row=18, column=4)


In [ ]:
root = Tk()
root.title('Verfügbarkeitslisten')
root.geometry('800x550')
root.config(bg='#eeeee4')
    
wozu = Label(root, text='Hallo DM!\n Mit diesem Programm werden die Verfügbarkeitslisten erstellt.',
             bg='#eeeee4', font=('Ink free', 15))
wozu.grid(row=1, column=1, columnspan=10)

# Auswählen der Datei
text_auswahl = Label(root, text = '1. Wähle bitte die Datei aus',
                     bg = '#eeeee4', font=('Ink free',11,'bold'))
text_auswahl.grid(row=2, column=1, columnspan=2)
browse_button = Button(root, text = 'Browse', bg = '#869287', font=('Ink free', 9, 'bold'),
                       command = select_file)
browse_button.grid(row=2, column=3)


# Dodo-Logo
frame = Frame(root, width=10, height=10)
frame.grid(row=1, column=0)
img = ImageTk.PhotoImage(Image.open("O:\Forschung & Entwicklung\Allgemein\Vorlagen\Abbildungen\Dodo\dodo-dancing_ohne Hintergrund_ohne Schatten.png").resize((100,100)), 
                 master=root,)
label = Label(frame, image=img, bg='#eeeee4')
label.pack()

root.mainloop()